# Prediction Model For Mid-Game Data

In [2]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pickle
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV

In [4]:
data = pd.read_csv('../../data/cleaned/all_mid_game_data_1103.csv')
data.head()

,SERVER,BTOP_CHAMP_ID,BJGL_CHAMP_ID,BMID_CHAMP_ID,BBOT_CHAMP_ID,BSUP_CHAMP_ID,BBAN1,BBAN2,BBAN3,BBAN4,...,RBAN1,RBAN2,RBAN3,RBAN4,RBAN5,FIRST_BLOOD,FIRST_DRAG,FIRST_HERALD,FIRST_TOWER,WINNER
0,EUROPE WEST,150,246,268,15,497,62,107,119,114,...,350,119,777,78,429,0,1,0,1,1
1,EUROPE WEST,114,254,79,498,497,555,-1,-1,103,...,157,432,777,119,350,0,1,1,1,1
2,EUROPE WEST,150,78,517,523,888,432,119,555,55,...,92,117,62,103,131,1,1,0,1,0
3,EUROPE WEST,79,80,4,18,235,517,20,350,122,...,517,421,67,119,163,0,1,0,0,1
4,EUROPE WEST,122,80,517,96,432,67,350,20,29,...,39,200,78,119,26,0,1,0,0,0


In [7]:
X = data.drop(columns=['SERVER', 'WINNER'])
y = data['WINNER']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=15)

In [8]:
feature_union = FeatureUnion([('pca', PCA()), 
                              ('select_best', SelectKBest())])

In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC

pipeline = Pipeline([
    ('features', feature_union),
    ('clf', LogisticRegression())
])

pipeline.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('pca', PCA()),
                                                ('select_best',
                                                 SelectKBest())])),
                ('clf', LogisticRegression())])

In [10]:
pipeline.score(X_test, y_test)

0.7341389728096677

In [11]:
param_grid = {"features__pca__n_components": [1, 2, 3, 4, 5],
                "features__select_best__k": [3, 4, 5, 6],
                'clf__penalty': ['l1','l2'],
                'clf__C': [0.001,0.01,0.1,1,10,100,1000]}

grid_search = GridSearchCV(pipeline, param_grid, verbose=5, n_jobs=-1, refit=True)    

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 280 candidates, totalling 1400 fits
[CV 2/5] END clf__C=0.001, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=4;, score=nan total time=   0.0s
[CV 3/5] END clf__C=0.001, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=3;, score=nan total time=   0.0s
[CV 4/5] END clf__C=0.001, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=3;, score=nan total time=   0.0s
[CV 2/5] END clf__C=0.001, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=3;, score=nan total time=   0.0s
[CV 1/5] END clf__C=0.001, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=4;, score=nan total time=   0.0s
[CV 1/5] END clf__C=0.001, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=3;, score=nan total time=   0.0s
[CV 3/5] END clf__C=0.001, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=4;, score=nan total time=   0.0s
[CV 5

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

[CV 4/5] END clf__C=0.01, clf__penalty=l2, features__pca__n_components=3, features__select_best__k=6;, score=0.669 total time=   0.0s
[CV 5/5] END clf__C=0.01, clf__penalty=l2, features__pca__n_components=1, features__select_best__k=4;, score=0.766 total time=   0.0s
[CV 1/5] END clf__C=0.1, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=3;, score=nan total time=   0.0s
[CV 4/5] END clf__C=0.01, clf__penalty=l2, features__pca__n_components=3, features__select_best__k=3;, score=0.714 total time=   0.0s
[CV 3/5] END clf__C=0.01, clf__penalty=l2, features__pca__n_components=5, features__select_best__k=4;, score=0.721 total time=   0.0s
[CV 2/5] END clf__C=0.1, clf__penalty=l1, features__pca__n_components=1, features__select_best__k=3;, score=nan total time=   0.0s
[CV 2/5] END clf__C=0.01, clf__penalty=l2, features__pca__n_components=4, features__select_best__k=5;, score=0.703 total time=   0.0s
[CV 1/5] END clf__C=0.01, clf__penalty=l2, features__pca__n_componen

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

[CV 3/5] END clf__C=1, clf__penalty=l1, features__pca__n_components=5, features__select_best__k=5;, score=nan total time=   0.0s
[CV 1/5] END clf__C=0.1, clf__penalty=l2, features__pca__n_components=4, features__select_best__k=4;, score=0.671 total time=   0.0s
[CV 1/5] END clf__C=1, clf__penalty=l1, features__pca__n_components=4, features__select_best__k=4;, score=nan total time=   0.0s
[CV 4/5] END clf__C=0.1, clf__penalty=l2, features__pca__n_components=2, features__select_best__k=5;, score=0.701 total time=   0.0s
[CV 2/5] END clf__C=1, clf__penalty=l1, features__pca__n_components=4, features__select_best__k=4;, score=nan total time=   0.0s
[CV 5/5] END clf__C=0.1, clf__penalty=l2, features__pca__n_components=5, features__select_best__k=5;, score=0.753 total time=   0.0s
[CV 3/5] END clf__C=1, clf__penalty=l1, features__pca__n_components=4, features__select_best__k=4;, score=nan total time=   0.0s
[CV 2/5] END clf__C=0.1, clf__penalty=l2, features__pca__n_components=1, features__se

...

[CV 4/5] END clf__C=100, clf__penalty=l1, features__pca__n_components=5, features__select_best__k=5;, score=nan total time=   0.0s
[CV 5/5] END clf__C=100, clf__penalty=l2, features__pca__n_components=2, features__select_best__k=6;, score=0.773 total time=   0.0s
[CV 2/5] END clf__C=100, clf__penalty=l1, features__pca__n_components=4, features__select_best__k=4;, score=nan total time=   0.0s
[CV 4/5] END clf__C=100, clf__penalty=l2, features__pca__n_components=1, features__select_best__k=4;, score=0.688 total time=   0.0s
[CV 3/5] END clf__C=10, clf__penalty=l2, features__pca__n_components=4, features__select_best__k=3;, score=0.773 total time=   0.0s
[CV 3/5] END clf__C=100, clf__penalty=l1, features__pca__n_components=4, features__select_best__k=4;, score=nan total time=   0.0s
[CV 4/5] END clf__C=10, clf__penalty=l2, features__pca__n_components=5, features__select_best__k=5;, score=0.675 total time=   0.0s
[CV 4/5] END clf__C=100, clf__penalty=l1, features__pca__n_components=4, feat

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

[CV 5/5] END clf__C=1000, clf__penalty=l2, features__pca__n_components=3, features__select_best__k=5;, score=0.766 total time=   0.0s
[CV 3/5] END clf__C=100, clf__penalty=l2, features__pca__n_components=5, features__select_best__k=6;, score=0.760 total time=   0.0s
[CV 1/5] END clf__C=1000, clf__penalty=l2, features__pca__n_components=3, features__select_best__k=6;, score=0.671 total time=   0.0s
[CV 3/5] END clf__C=1000, clf__penalty=l2, features__pca__n_components=3, features__select_best__k=6;, score=0.747 total time=   0.0s
[CV 2/5] END clf__C=1000, clf__penalty=l2, features__pca__n_components=3, features__select_best__k=6;, score=0.697 total time=   0.0s
[CV 4/5] END clf__C=100, clf__penalty=l2, features__pca__n_components=5, features__select_best__k=6;, score=0.701 total time=   0.0s
[CV 4/5] END clf__C=1000, clf__penalty=l2, features__pca__n_components=3, features__select_best__k=6;, score=0.701 total time=   0.0s
[CV 2/5] END clf__C=1000, clf__penalty=l2, features__pca__n_comp

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('pca',
                                                                        PCA()),
                                                                       ('select_best',
                                                                        SelectKBest())])),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'clf__penalty': ['l1', 'l2'],
                         'features__pca__n_components': [1, 2, 3, 4, 5],
                         'features__select_best__k': [3, 4, 5, 6]},
             verbose=5)

In [12]:
grid_search.score(X_test, y_test)

0.7250755287009063